In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kalapas/test.csv
/kaggle/input/kalapas/train.csv
/kaggle/input/kalapas/simple_submission.csv


# Prepare data V1

In [2]:
import pandas as pd
import numpy as np
from fastai import *
from fastai.tabular import *
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [46]:
train_df = pd.read_csv('/kaggle/input/kalapas/train.csv')
test_df = pd.read_csv('/kaggle/input/kalapas/test.csv')
label = train_df['label']

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
DATE = ["Field_{}".format(i) for i in [5, 6, 7, 8, 9, 11, 15, 25, 32, 33,34, 35, 40]]
DATETIME = ["Field_{}".format(i) for i in [1,2,43,44]]
date_need_correct = ['Field_34', 'ngaySinh']
start_end_date = [f'{cat}_startDate' for cat in ['F', 'E', 'C', 'G', 'A'] ] + [f'{cat}_endDate' for cat in ['F', 'E', 'C', 'G', 'A'] ]
date_names = DATE + DATETIME + start_end_date

In [48]:
def str2date(s):
  if s!=s:
    return np.nan
  try:
    date_obj = datetime.strptime(str(s),'%m/%d/%Y')
  except:
    date_obj = datetime.strptime(str(s), '%Y-%m-%d')
  return date_obj
def correct_34_ngaysinh(s):
    if s != s:
        return np.nan
    try:
        s = int(s)
    except ValueError:
        s = s.split(" ")[0]
        
    return datetime.strptime(str(s)[:6], "%Y%m")
def datetime_normalize(s):
    if s != s:
        return np.nan
    
    s = s.split(".")[0]
    if s[-1] == "Z":
        s = s[:-1]
        
    date, time = s.split("T")
    datetime_obj = datetime.strptime(s, "%Y-%m-%dT%H:%M:%S")
    return datetime.date(datetime_obj)
def process_datetime(df):
  for col in DATE:
    if col == 'Field_34':
      continue
    df[col] = df[col].apply(str2date)
  df['Field_34'] = df['Field_34'].apply(correct_34_ngaysinh)
  df['ngaySinh'] = df['ngaySinh'].apply(correct_34_ngaysinh)
  for col in DATETIME:
    df[col] = df[col].apply(datetime_normalize)
  for col in start_end_date:
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
  return df
def make_new_features(df):
  for cat in ['F', 'E', 'C', 'G', 'A']:
    df[f'{cat}_span'] = df[f'{cat}_endDate'] - df[f'{cat}_startDate']
  def take_day(d):
    return d.days
  for cat in ['F', 'E', 'C', 'G', 'A']:
    df[f'{cat}_span'] = df[f'{cat}_span'].apply(take_day)
  return df


#processing datetime data in df
train_df = process_datetime(train_df)
test_df = process_datetime(test_df)
train_df = make_new_features(train_df)
test_df = make_new_features(test_df)




In [49]:
#process diaChi and maCv columns:
def str_normalize(s):
  s = str(s).strip().lower()
  s = re.sub(' +', " ", s)
  return s
def job_category(x):
  if type(x) == str:
    if "công nhân" in x or "cnv" in x or "cn" in x or "may công nghiệp" in x or "lao động" in x\
    or "thợ" in x or "coõng nhaõn trửùc tieỏp maựy may coõng nghieọp" in x or "c.n" in x or "lđ" in x:
        return "CN"
    elif "giáo viên" in x or "gv" in x or "gíao viên" in x:
        return "GV"
    elif "nhân viên" in x or "kế toán" in x or "cán bộ" in x or "nv" in x or "cb" in x or "nhõn viờn" in x:
        return "NV"
    elif "tài xế" in x or "lái" in x or "tài xê" in x:
        return "TX"
    elif "quản lý" in x or "phó phòng" in x or "hiệu phó" in x:
        return "QL"
    elif "undefined" in x:
        return "missing"
    elif "giám đốc" in x or "hiệu trưởng" in x:
        return "GĐ"
    elif "phục vụ" in x:
        return "PV"
    elif "chuyên viên" in x:
        return  "CV"
    elif "bác sĩ" in x or "dược sĩ" in x or "y sĩ" in x or "y sỹ" in x:
        return "BS"
    elif "y tá" in x:
        return "YT"
    elif "hộ sinh" in x:
        return "HS"
    elif "chủ tịch" in x:
        return "CT"
    elif "bếp" in x:
        return "ĐB"
    elif "sư" in x:
        return "KS"
    elif "dưỡng" in x:
        return "ĐD"
    elif "kỹ thuật" in x or "kĩ thuật" in x:
        return "KTV"
    elif "diễn viên" in x:
        return "DV"
    else:
        return "missing"
  else:
    return x    
  
def process_diaChi_maCv(df):
    df["maCv"] = df["maCv"].apply(str_normalize).apply(job_category).astype("category")
    return df
train_df = process_diaChi_maCv(train_df)
test_df = process_diaChi_maCv(test_df)

In [50]:
#combine 2 gender features as 1
def combine_gender(s):
  x, y = s
  
  if x != x and y != y:
      return np.nan
  
  if x != x:
      return y.lower()
  
  return x.lower()

def process_gender(df):
  df["gender"] = df[["gioiTinh", "info_social_sex"]].apply(combine_gender, axis=1).astype("category")
  df = df.drop(['gioiTinh', 'info_social_sex'],axis = 1)
  return df
train_df = process_gender(train_df)
test_df = process_gender(test_df)

In [51]:
def process_misc(df):        
  df["subscriberCount"].replace(0, np.nan, inplace=True)
  df["friendCount"].replace(0, np.nan, inplace=True)
  
  df["Field_13"] = df["Field_13"].apply(lambda x: 1 if x == x else 0)
  df["Field_38"] = df["Field_38"].map({0: 0.0, 1: 1.0, "DN": np.nan, "TN": np.nan, "GD": np.nan})
  df["Field_62"] = df["Field_62"].map({"I": 1, "II": 2, "III": 3, "IV": 4, "V": 5, "Ngoài quốc doanh Quận 7": np.nan})
  df["Field_47"] = df["Field_47"].map({"Zezo": 0, "One": 1, "Two": 2, "Three": 3, "Four": 4})
  
  df["Field_27"] = df["Field_27"].replace({0.0: np.nan})
  df["Field_28"] = df["Field_28"].replace({0.0: np.nan})
      
  for col in df.columns:
    if df[col].dtype.name == "object":
      df[col] = df[col].apply(str_normalize).astype("category")
          
  return df
train_df = process_misc(train_df)
test_df = process_misc(test_df)

# process string features


In [54]:
def feature_diaChi(s):
    if type(s) == str:
        if 'hcm' in s or 'hồ chí minh' in s:
            return 'hcm'
        elif 'tỉnh' in s:
            return 'tinh'
        elif 'ha noi' in s or 'hà nội' in s:
            return 'hanoi'
        else:
            return 'tinh'
        
train_df['diaChi'] =train_df['diaChi'].apply(feature_diaChi)

tinh     48186
hcm       3280
hanoi     1564
Name: diaChi, dtype: int64

In [36]:
def field_46(s):
    if type(s) == str:
        
        if 'thất nghiệp' in s:
            return 'thatnghiep'
        elif s == 'nan':
            return s
        elif 'cty' in s or 'công ty' in s or 'c.ty' in s:
            return 'cty'
        elif 'đại lý' in s:
            return 'daily'
        elif 'đại học' in s or 'cđ' in s:
            return 'daihoc'
        elif 'hộ gia đình' in s or 'hgđ':
            return 'giadinh'
        else:
            return s
train_df['Field_46'] =  train_df['Field_46'].apply(field_46)

In [45]:
def Field_48(s):
    if type(s) == str:
        if 'hcm' in s or 'hồ chí minh' in s:
            return 'hcm'
        elif 'tỉnh' in s:
            return 'tinh'
        elif 'ha noi' in s or 'hà nội' in s:
            return 'hanoi'
        else:
            return s
        
train_df['Field_48'].apply(feature_diaChi).value_counts()

tinh     49168
hcm       1959
hanoi     1903
Name: Field_48, dtype: int64

In [9]:
#Drop some field we do not need
# drop some fields we do not need (homeTown is optionally)
DROP = ["namSinh", ''] + \
        [f"Field_{c}" for c in [14, 16, 17,18, 24, 26, 30, 31, 37, 52, 57]]
train_df.drop(DROP, axis = 1,inplace = True)
test_df.drop(DROP, axis =1, inplace = True)

In [20]:
train_df.head()

,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,Field_10,Field_11,Field_12,Field_13,Field_15,Field_18,Field_19,Field_20,Field_21,Field_22,Field_23,Field_25,Field_27,Field_28,Field_29,Field_32,Field_33,Field_34,Field_35,ngaySinh,diaChi,Field_36,Field_38,Field_39,Field_40,Field_41,Field_42,Field_43,Field_44,Field_45,Field_46,Field_47,Field_48,Field_49,Field_50,Field_51,Field_53,Field_54,Field_55,Field_56,Field_58,Field_59,Field_60,Field_61,Field_62,Field_63,Field_64,Field_65,Field_66,Field_67,Field_68,maCv,Field_69,Field_70,Field_71,Field_72,Field_73,Field_74,Field_75,Field_76,Field_77,friendCount,subscriberCount,currentLocationLocationId,currentLocationLatitude,currentLocationLongitude,homeTownLocationId,homeTownLatitude,homeTownLongitude,data.basic_info.locale,currentLocationCity,currentLocationCountry,currentLocationName,currentLocationState,homeTownCity,homeTownCountry,homeTownName,homeTownState,topFriends,numOrg,F_numOrg,F_numQuery,F_startDate,F_endDate,E_numOrg,E_numQuery,E_startDate,E_endDate,C_numOrg,C_numQuery,C_startDate,C_endDate,G_numOrg,G_numQuery,G_startDate,G_endDate,A_numOrg,A_numQuery,A_startDate,A_endDate,summary_6m,summary_3m,summary_1m,summary_1w,partner0_A,partner0_B,partner0_C,partner0_D,partner0_E,partner0_F,partner0_G,partner0_H,partner0_K,partner0_L,partner1_A,partner1_B,partner1_C,partner1_D,partner1_E,partner1_F,partner1_G,partner1_H,partner1_K,partner1_L,partner2_A,partner2_B,partner2_C,partner2_D,partner2_E,partner2_F,partner2_G,partner2_H,partner2_K,partner2_L,partner3_A,partner3_B,partner3_C,partner3_D,partner3_E,partner3_F,partner3_G,partner3_H,partner3_K,partner3_L,partner4_A,partner4_B,partner4_C,partner4_D,partner4_E,partner4_F,partner4_G,partner4_H,partner4_K,partner4_L,partner5_A,partner5_B,partner5_C,partner5_D,partner5_E,partner5_F,partner5_G,partner5_H,partner5_K,partner5_L,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82,F_span,E_span,C_span,G_span,A_span,gender
0,2019-07-24,2019-07-31,1.0,gh,2018-12-27,2018-12-27,2019-01-01,2019-07-31,2018-12-27,1.0,2019-07-31,g8,1,NaT,trung tâm kinh doanh tiền mặt,0.0,4258600.0,4.5,1.0,16.0,2018-12-27,NaN,NaN,0.0,2018-12-28,2022-09-01,2018-01-01,2019-07-31,1991-06-01,NaN,dn,0.0,1.0,2019-01-01,1.0,1.0,2017-01-03,2019-11-12,tf4354f,công ty tài chính tnhh mb shinsei,4.0,"tầng 9, tòa nhà mb, số 21 cát linh, đống đa, hn","tầng 12, tòa nhà tnr, số 54a nguyễn chí thanh,...",1.0,1.0,0.0,vn,tf,ngoài quốc doanh quận đống đa,0.0,5.0,3.0,khối dn ngoài quốc doanh,1.0,2.0,1.0,vùng 1,g,118410.0,0107349019,NV,0.00,0.49,0.245,0.49,NaN,NaN,5.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,5.0,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,2.0,2.0,2019-06-11,2019-08-17,2.0,1.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,cb1,1.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,67.0,male
1,nan,nan,NaN,nan,NaT,NaT,NaT,NaT,NaT,NaN,NaT,nan,0,NaT,nan,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,nan,NaN,NaN,NaT,NaN,NaN,nan,nan,nan,nan,NaN,nan,nan,NaN,NaN,NaN,nan,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,NaN,nan,nan,NaN,nan,missing,NaN,NaN,NaN,NaN,1.0,704.0,6.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN,1.0,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,NaN,NaN,NaT,NaT,1.0,1.0,2019-07-17,2019-07-17,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,1.0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,0.0,NaN
2,2019-01-17,2019-01-17,2.0,t1,2019-01-17,2019-01-17,2019-01-01,2019-12-31,2019-01-17,1.0,NaT,nan,1,2019-01-17,nan,0.0,5000000.0,4.5,1.0,10.0,2019-01-17,NaN,NaN,0.0,2019

nan           27466
cty           14902
giadinh        8388
daily          1091
thatnghiep      875
daihoc          308
Name: Field_46, dtype: int64

In [39]:
train_df['Field_48'].value_counts()

nan                                                                                                                                           29457
153 xô viết nghệ tĩnh quận bình thạnh                                                                                                           377
162 tô hiệu, hà cầu, hà đông, hà nội                                                                                                             99
bình dương                                                                                                                                       98
128 trần quang khải, p. tân định, q.1                                                                                                            87
tầng 2 tòa nhà ree tower số 9 đoàn văn bơ, p12, q4, tp.hcm                                                                                       82
kcn yên bình 1, phường đồng tiến, thị xã phổ yên, tỉnh thái nguyên                                              

In [41]:
train_df['diaChi'].value_counts()

Series([], Name: diaChi, dtype: int64)